In [1]:
import cv2
import os
import numpy as np
cascPathface = os.path.dirname(
    cv2.__file__) + "/data/haarcascade_frontalface_alt2.xml"
faceCascade = cv2.CascadeClassifier(cascPathface)

video_capture = cv2.VideoCapture(0)

# Define the codec and create a VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 20.0, (640, 480))

# Load the image and resize it to the face region size
img = cv2.imread('mask.png')
img = cv2.resize(img, (0, 0), fx=7, fy=7)

while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray,
                                         scaleFactor=1.1,
                                         minNeighbors=5,
                                         minSize=(60, 60),
                                         flags=cv2.CASCADE_SCALE_IMAGE)
    for (x,y,w,h) in faces:
        img_resized = cv2.resize(img, (w, h))

        # Create a mask of the face region
        mask = np.zeros(frame.shape[:2], dtype=np.uint8)
        mask[y:y+h, x:x+w] = 120

        # Use bitwise-and to apply the image to the face region
        masked_img = cv2.bitwise_and(img_resized, img_resized, mask=mask[y:y+h, x:x+w])
        frame[y:y+h, x:x+w] = masked_img

    # Write the frame to the output video file
    out.write(frame)

    # Display the resulting frame
    cv2.imshow('Face Video', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release everything when done
video_capture.release()
out.release()
cv2.destroyAllWindows()
